In [27]:
import pandas as pd

# NLTK - DETECTOR DE SPAM

In [28]:
df = pd.read_csv('DataBase.csv')
df

,response,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [29]:
df.groupby('response').describe()

message                                                            \
           count unique                                                top   
response                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    653  Please call our customer service representativ...   

               
         freq  
response       
ham        30  
spam        4

# TRATAMENTO DOS DADOS
- Temos que tratar e limpar os dados para melhorar a precisão do modelo, uma forma de fazer isso é removendo a pontuação (" ! ", " ? ", " ; "...) e dos textos e também remover palavras desnecessárias desnecessárias, também chamadas de stopwords (" i ", " You", " que ", " isso "...)

In [30]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/klauss/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [31]:
import string
from nltk.corpus import stopwords

In [32]:
def obterRelevantes(texto):
    semPontuacao = [char for char in texto if char not in string.punctuation] 
    semPontuacao = ''.join(semPontuacao) 
    semPontuacao = semPontuacao.split()
    return [palavra for palavra in semPontuacao if palavra.lower() not in stopwords.words('english')]


df['message'].head(5).apply(obterRelevantes)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: message, dtype: object

- Criando o objeto sacoPalavras que usa o CountVectorizer para contar as palavras relevantes selecionado pela função obterRelevantes que criamos e inserimos no atributo analyzer. Após isso
ajustamos esse objeto à coluna sacoPalavras, onde o processamento é feito para cada linha da coluna

In [33]:
from sklearn.feature_extraction.text import CountVectorizer

In [34]:
sacoPalavras = CountVectorizer(analyzer=obterRelevantes)
sacoPalavras.fit(df['message'])

list(sacoPalavras.vocabulary_.items())[:5]

[('Go', 2060),
 ('jurong', 7555),
 ('point', 8917),
 ('crazy', 5769),
 ('Available', 1110)]

- Armazenando o saco de palavras para as mensagens usando o método de transformador

In [35]:
mensagemSacoPalavras = sacoPalavras.transform(df['message'])
mensagemSacoPalavras

<5572x11425 sparse matrix of type '<class 'numpy.int64'>'
	with 50548 stored elements in Compressed Sparse Row format>

- Aplicando o transformador para o processamento do texto pelo ajuste do saco de palavras

In [36]:
from sklearn.feature_extraction.text import TfidfTransformer

In [37]:
transformador = TfidfTransformer().fit(mensagemSacoPalavras)
mensagemTransformador = transformador.transform(mensagemSacoPalavras)
mensagemTransformador

<5572x11425 sparse matrix of type '<class 'numpy.float64'>'
	with 50548 stored elements in Compressed Sparse Row format>

# MODELO DE PREVISÃO
- Usando o modelo "Naive Bayes" para detectar o spam e ajustá-lo usando os dados na forma " tfidf " que criamos nas linhas de código acima

In [38]:
from sklearn.naive_bayes import MultinomialNB

In [39]:
spamDetector = MultinomialNB().fit(mensagemTransformador, df['response'])
spamDetector

MultinomialNB()

- Usando o spamDetector que criamos para fazer previsões de algumas mensagens que já sabemos quais são as respostas pelo documento

In [40]:
mensagem = 'I HAVE A DATE ON SUNDAY WITH WILL!!'
mensagemTransformada = sacoPalavras.transform([mensagem])
TFIDF = transformador.transform(mensagemTransformada)
resposta = spamDetector.predict(TFIDF)[0]

print('Mensagem: ', mensagem)
print('Categoria: ', resposta)

Mensagem:  I HAVE A DATE ON SUNDAY WITH WILL!!
Categoria:  ham


In [41]:
mensagem = 'Thanks for your subscription to Ringtone UK your mobile will be charged £5/month'
mensagemTransformada = sacoPalavras.transform([mensagem])
TFIDF = transformador.transform(mensagemTransformada)
resposta = spamDetector.predict(TFIDF)[0]

print('Mensagem: ', mensagem)
print('Categoria: ', resposta)

Mensagem:  Thanks for your subscription to Ringtone UK your mobile will be charged £5/month
Categoria:  spam


# SALVANDO O MODELO

In [42]:
from joblib import dump

# Salvar o vetorizador
dump(sacoPalavras, 'sacoPalavras.joblib')

# Salvar o transformador TF-IDF, se você estiver usando
dump(transformador, 'tfidf_transformer.joblib')

# Salvar o modelo de detecção de spam
dump(spamDetector, 'spam_detector_model.joblib')

['spam_detector_model.joblib']

In [43]:
from joblib import load

# Carregar o vetorizador
sacoPalavras = load('sacoPalavras.joblib')

# Carregar o transformador TF-IDF
transformador = load('tfidf_transformer.joblib')

# Carregar o modelo de detecção de spam
spamDetector = load('spam_detector_model.joblib')

In [44]:
mensagem = 'Thanks for your subscription to Ringtone UK your mobile will be charged £5/month'
mensagemTransformada = sacoPalavras.transform([mensagem])
TFIDF = transformador.transform(mensagemTransformada)
resposta = spamDetector.predict(TFIDF)[0]

print('Mensagem: ', mensagem)
print('Categoria: ', resposta)

Mensagem:  Thanks for your subscription to Ringtone UK your mobile will be charged £5/month
Categoria:  spam
